In [1]:
import pandas as pd
from pymongo import MongoClient
from datetime import datetime, timedelta
import concurrent.futures
import sys
import os.path
import bson as b


from pyspark.sql.types import *

from pyspark.sql import types as T 
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from datetime import datetime, timedelta 
import pyspark
from pyspark.sql import SQLContext  

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
 

def getSpark():
    conf = pyspark.SparkConf() \
                .set("spark.jars.packages","org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
                .set('spark.sql.caseSensitive', True) \
                .setMaster("local") \
                .setAppName("sparkaApp") \
                .setAll([("spark.driver.memory", "4g"), ("spark.executor.memory", "6g")])
                
    sc = SparkContext(conf=conf)
    spark = SparkSession(sc)
    return spark

local = "True"
if local == "True":
    spark = getSpark()
    
uri_string = "mongodb://localhost:27017/?readPreference=primary&appname=MongoDB%20Compass&directConnection=true&ssl=false"
dbName = 'local'
colName = 'STP'
client = MongoClient(uri_string)
database = client.get_database(dbName)
col = database.get_collection(colName)


In [ ]:
 #.option('spark.mongodb.input.sampleSize', sampleSize) \
 #.option('spark.mongodb.input.partitioner', "MongoSamplePartitioner") \
 #.option('spark.mongodb.input.partitionerOptions.partitionSizeMB', partitionSizeMB) \
 #.option('spark.mongodb.input.partitionerOptions.samplesPerPartition', samplesPerPartition) \

#PYMONGO METHODS

In [3]:

def colCount():
    colCount =  col.find({},{"_id"}).count()
    return colCount

def colFind(limit=""):
    if limit == "":
        colList = [ x for x in col.find()]
    else:
        colList = [ x for x in col.find().limit(limit)]
    return(colList)

def colRawFind(limit=""):
    if limit == "":
        colList = [ x for x in col.find_raw_batches()]
    else:
        colList = [ b.decode_all(x) for x in col.find_raw_batches().limit(limit)]
    return(colList)

def colFindID():
    idList = [str(x).split("'",4)[3] for x in col.find({},{"_id":1}).limit(5)]
    return(idList)
    


#PYSPARK METHODS

In [2]:
DATA_SCHEMA = spark.read\
                .format( "com.mongodb.spark.sql.DefaultSource")\
                .option('spark.mongodb.input.uri', uri_string)\
                .option('spark.mongodb.input.sampleSize', 100)\
                .option("spark.mongodb.input.database", "local")\
                .option("spark.mongodb.input.collection", "local_test")\
                .load().schema

In [13]:
DATA_COLS = spark.read\
            .schema(DATA_SCHEMA)\
            .format( "com.mongodb.spark.sql.DefaultSource")\
            .option('spark.mongodb.input.uri', uri_string)\
            .option("spark.mongodb.input.database", "local")\
            .option("spark.mongodb.input.collection", "local_test")\
            .load().columns

['_id',
 'cnpj',
 'codigo_qualificacao_socio',
 'codigo_tipo_socio',
 'cpf_cnpj_socio',
 'nome_socio',
 'qualificacao_socio',
 'razao_social',
 'tipo_socio']

%md USO DE SQUEMA OTIMIZADO REDUZIU A QUERY EM 20%

In [15]:

   
DATA_IDS = DATA_COLS = spark.read\
            .schema(DATA_SCHEMA)\
            .format( "com.mongodb.spark.sql.DefaultSource")\
            .option('spark.mongodb.input.uri', uri_string)\
            .option("spark.mongodb.input.database", "local")\
            .option("spark.mongodb.input.collection", "local_test")\
            .load().select("_id")
DATA_IDS.count()      


#----5m2.4s  
DATA_IDS_SCHEMA = DATA_COLS = spark.read\
            .schema(DATA_SCHEMA)\
            .format( "com.mongodb.spark.sql.DefaultSource")\
            .option('spark.mongodb.input.uri', uri_string)\
            .option('spark.mongodb.input.sampleSize', 50)\
            .option("spark.mongodb.input.database", "local")\
            .option("spark.mongodb.input.collection", "local_test")\
            .load().select("_id").schema



#----4m1.1s 
DATA_IDS_S_FILTER = DATA_COLS = spark.read\
            .schema(DATA_IDS_SCHEMA)\
            .format( "com.mongodb.spark.sql.DefaultSource")\
            .option('spark.mongodb.input.uri', uri_string)\
            .option("spark.mongodb.input.database", "local")\
            .option("spark.mongodb.input.collection", "local_test")\
            .load()

DATA_IDS_S_FILTER.count()   

27420729

%md #metodos altenarivos de seleção

In [6]:


DATA_ =  spark.read\
            .schema(DATA_SCHEMA)\
            .format( "com.mongodb.spark.sql.DefaultSource")\
            .option('spark.mongodb.input.sampleSize', 10)\
            .option('spark.mongodb.input.uri', uri_string)\
            .option("spark.mongodb.input.database", "local")\
            .option("spark.mongodb.input.collection", "local_test")\
            .load().select(['_id',
                            'cnpj',
                            'codigo_qualificacao_socio',
                            'codigo_tipo_socio',
                            'cpf_cnpj_socio',
                            'nome_socio',
                            'qualificacao_socio',
                            'razao_social',
                            'tipo_socio']).show()


+--------------------+--------------+-------------------------+-----------------+--------------+--------------------+------------------+------------------+-------------+
|                 _id|          cnpj|codigo_qualificacao_socio|codigo_tipo_socio|cpf_cnpj_socio|          nome_socio|qualificacao_socio|      razao_social|   tipo_socio|
+--------------------+--------------+-------------------------+-----------------+--------------+--------------------+------------------+------------------+-------------+
|{615fbce78969fa8e...|00000000296376|                       10|                2|   ***760516**|DELANO VALENTIM D...|           Diretor|BANCO DO BRASIL SA|Pessoa Física|
|{615fbce78969fa8e...|00000000000353|                       16|                2|   ***470447**|RUBEM DE FREITAS ...|        Presidente|BANCO DO BRASIL SA|Pessoa Física|
|{615fbce78969fa8e...|00000000000353|                       10|                2|   ***431620**|GERSON EDUARDO DE...|           Diretor|BANCO DO BRASI

#PARALLEL METHODS

In [ ]:
def maxRunWorkerTask(ID=None):
    #TRANSFORMATION FUNC

    QUERY = db.client.get_database("local").get_collection("SPT").find({ 'match' : {'_id': ID}})     
    for x in QUERY:
        RESULT_LIST.append(x)
    

def maxRunWorker(max_workers=4, mode="submit"):    
    ID_LIST = colFindID()
    T0 = datetime.now()
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        if mode == "submit":
            for x in ID_LIST:
                executor.submit(maxRunWorkerTask, ID=x)
        else:
            executor.map(maxRunWorkerTask, ID_LIST)
        
    T1 = datetime.now()
    print(f" QUERY_TIME = {(T1-T0).seconds} seconds")
